In [44]:
import numpy as np
import pandas as pd
import json
import datetime
import glob
import gc
import os

In [45]:
folder="/home/jian/Projects/Big_Lots/Weather/Json_data/forecast/forcast_api_response_20180524/"

json_list_all=glob.glob(folder+"*.json")

In [46]:
inclusion_stores=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180514.xlsx",sheetname="Inclusion Stores",skiprows=1,dtype=str)
store_zips=pd.read_excel("/home/jian/Projects/Big_Lots/Other_Input/all_store_DMA.xlsx",dtype=str)[['location_id','zip']]
store_zips['zip']=store_zips['zip'].apply(lambda x: x.zfill(5))
inclusion_stores=pd.merge(inclusion_stores,store_zips,on="location_id",how="left")
inclusion_stores=inclusion_stores[['location_id','zip']].rename(columns={"zip":"zip_cd"})
inclusion_stores=inclusion_stores[inclusion_stores['location_id']!='1769']

In [47]:
df_files=pd.DataFrame({"file":json_list_all},index=range(len(json_list_all)))
df_files['Date']=df_files['file'].apply(lambda x: datetime.datetime.strptime(x[len(x)-15:len(x)-5],"%Y-%m-%d"))
df_files=df_files[df_files['Date']>=datetime.datetime(2018,2,4)]

In [48]:
'''
file=df_files['file'][0]
test=json.load(open(file,"r"))

len(test['36535']['list'])
'''

'\nfile=df_files[\'file\'][0]\ntest=json.load(open(file,"r"))\n\nlen(test[\'36535\'][\'list\'])\n'

In [49]:
all_weather_groups=[]
all_weather_desc=[]
df_missing_weather=pd.DataFrame()
for file in df_files['file']:
    response=json.load(open(file,"r"))
    for hour_index_forecast in range(40):
        for zip_cd in inclusion_stores['zip_cd']:
            try:        
                data_zip_weather=response[zip_cd]['list'][hour_index_forecast]['weather']
                for i in range(len(data_zip_weather)):            
                    weather_group=data_zip_weather[i]['main']
                    weather_desc=data_zip_weather[i]['description']



                    all_weather_groups=list(set(all_weather_groups+[weather_group]))
                    all_weather_desc=list(set(all_weather_desc+[weather_desc]))

            except:
                df_missing_weather=df_missing_weather.append(pd.DataFrame({"Date":file[len(file)-15:len(file)-5],"zips":zip_cd},index=[0]))
                # print("zip data not available: " + zip_cd+" | "+file[len(file)-15:len(file)-5])

In [50]:
all_weather_groups

['Clouds', 'Clear', 'Snow', 'Rain']

In [51]:
all_weather_desc

['very heavy rain',
 'heavy intensity rain',
 'scattered clouds',
 'snow',
 'few clouds',
 'broken clouds',
 'overcast clouds',
 'light rain',
 'moderate rain',
 'clear sky',
 'heavy snow',
 'light snow']